In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from utils.helpers import *
from utils.prediction import *
from utils.preprocess import *
from utils.cross_validation import *

In [3]:
TRAIN_PATH = './data/train.csv'
TEST_PATH = './data/test.csv'

In [4]:
lambda_ = 1e-3
degree = 7
learning_rate = 0.1
max_iter = 1000
optimizer = 'gd'
k_fold = 5
seed = 20221031
batch_size = 8

In [5]:
y_raw_tr, tx_raw_tr, ids_tr = load_csv_data(TRAIN_PATH)
_, tx_raw_te, ids_te = load_csv_data(TEST_PATH)

In [6]:
y_tr = process_y(y_raw_tr)
tx_tr = process_tx(tx_raw_tr)
tx_te = process_tx(tx_raw_te)
print(y_tr.shape)
print(tx_tr.shape)
print(tx_te.shape)

(250000, 1)
(250000, 29)
(568238, 29)


In [7]:
# cross validation
k_indices = build_k_indices(y_tr, k_fold, seed)
tx_tr, tx_dev, y_tr, y_dev = cross_validation_dataset(y_tr, tx_tr, k_indices, k=k_fold-1)
print(tx_tr.shape)
print(tx_dev.shape)
print(y_tr.shape)
print(y_dev.shape)

(200000, 29)
(50000, 29)
(200000, 1)
(50000, 1)


In [8]:
# split datasets to different jet nums
# and remove columns with missing values for each jet num
tx_train_list, y_tr_list = split_jet_num(tx_tr, y_tr)
tx_dev_list, y_dev_list = split_jet_num(tx_dev, y_dev)

In [9]:
# add polynomial features
for i in range(3):
    tx_train_list[i] = build_poly(tx_train_list[i], degree)
    tx_dev_list[i] = build_poly(tx_dev_list[i], degree)

In [10]:
for i in range(3):
    print(tx_train_list[i].shape, tx_dev_list[i].shape)

(79917, 119) (19996, 119)
(62257, 147) (15287, 147)
(57826, 196) (14717, 196)


In [12]:
means = [0, 0, 0]
stds = [0, 0, 0]
for i in range(3):
    tx_train_list[i], tx_dev_list[i], means[i], stds[i] = normalization(
        tx_train_list[i],
        tx_dev_list[i]
    )

In [12]:
for i in range(3):
    print(tx_train_list[i].shape, tx_dev_list[i].shape, means[i].shape, stds[i].shape)

NameError: name 'means' is not defined

In [23]:
def reg_logistic_regression_plot(y_tr, tx_tr, y_dev, tx_dev, lambda_, initial_w, max_iters, gamma, batch_size=8, optimizer='gd'):
    """Regularized logistic regression using gradient descent
    or SGD (y ∈ {0, 1}, with regularization term λ|w|2)

    Args:
        y_tr: numpy array of shape=(N_tr, 1)
        tx_tr: numpy array of shape=(N_tr, D)
        y_dev: numpy array of shape=(N_dev, 1)
        tx_dev: numpy array of shape=(N_dev, D)
        lambda_: a scalar denoting the regularization term
        initial_w: numpy array of shape=(D, 1). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of SGD
        gamma: a scalar denoting the stepsize
        batch_size: mini batch size. default 8.
        optimizer: 'gd' (batch sgd), 'ada' (adagrad), and 'adam'. default 'gd'.

    Returns:
        w: the best weight vector of shape (D, 1) for validation
        train_loss: the corresponding mse loss
        dev_loss: the corresponding mse loss
    """

    # Define parameters to store w and loss
    w = initial_w
    ws = [initial_w]
    train_losses = [compute_ce(y_tr, tx_tr, w)]
    dev_losses = [compute_ce(y_dev, tx_dev, w)]
    grad_square_sum = 0
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-9
    m = 0
    v = 0

    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(
            y_tr, tx_tr, batch_size=batch_size, num_batches=1
        ):
            # compute gradient
            grad = logistic_reg_gradient(y_batch, tx_batch, w)

            if optimizer == 'gd':
                # update w by gradient
                w = w - gamma * (grad + 2 * lambda_ * w)
            elif optimizer == 'ada':
                grad_square_sum += grad**2
                adagrad = grad / np.sqrt(n_iter+1)
                w = w - gamma * (adagrad + 2 * lambda_ * w)
            elif optimizer == 'adam':
                m = beta1*m + (1-beta1)*grad
                v = beta2*v + (1-beta2)*grad**2
                m_db = m / (1-beta1**(n_iter+1))
                v_db = v / (1-beta2**(n_iter+1))
                adam_grad = m_db / (np.sqrt(v_db) + eps)
                w = w - gamma * (adam_grad + 2 * lambda_ * w)

            # compute loss
            loss = compute_ce(y_tr, tx_tr, w)

            # store w and loss
            ws.append(w)
            train_losses.append(loss)

            # compute dev loss
            dev_losses.append(compute_ce(y_dev, tx_dev, w))
    
    index = np.argmin(dev_losses)
    plt.plot(np.arange(max_iters+1), train_losses, label='training_loss')
    plt.plot(np.arange(max_iters+1), dev_losses, label='dev_loss')
    plt.legend()
    plt.xlabel('iters')
    plt.ylabel('loss')
    plt.title('loss')
    plt.savefig(str(tx_tr.shape[1]))
    plt.clf()
    return ws[index], train_losses[index], dev_losses[index]

In [25]:
train_losses = []
dev_losses = []
ws = []
y_tr_pred, y_tr_true = np.empty((0, 1)), np.empty((0, 1))
y_dev_pred, y_dev_true = np.empty((0, 1)), np.empty((0, 1))

for i in range(len(tx_train_list)):
    train_losses = []
    dev_losses = []
    w_list = []
    lambda_list = [i for i in np.logspace(-6, 2, 9)]

    y_tr = y_tr_list[i]
    tx_tr_fe = tx_train_list[i]
    y_dev = y_dev_list[i]
    tx_dev_fe = tx_dev_list[i]

    for lambda_ in lambda_list:
        initial_w = np.random.rand(tx_tr_fe.shape[1], 1)
        w, train_loss, dev_loss = reg_logistic_regression_plot(
            y_tr, tx_tr_fe,
            y_dev, tx_dev_fe,
            lambda_,
            initial_w,
            max_iter*3,
            0.001,
            batch_size=tx_tr_fe.shape[0],
            optimizer=optimizer
        )
        train_losses.append(train_loss)
        dev_losses.append(dev_loss)
        w_list.append(w)

    cross_validation_visualization(lambda_list, train_losses, dev_losses, i)
    index = np.argmin(dev_losses)
    best_lambda = lambda_list[index]
    best_w = w_list[index]
    print("The best lambda for PRI_JET_NUM = {} is {}.".format(i, best_lambda))

    y_tr_pred = np.vstack((y_tr_pred, predict_logistic(tx_tr_fe, best_w)))
    y_dev_pred = np.vstack((y_dev_pred, predict_logistic(tx_dev_fe, best_w)))
    y_tr_true = np.vstack((y_tr_true, y_tr))
    y_dev_true = np.vstack((y_dev_true, y_dev))
    ws.append(best_w)

The best lambda for PRI_JET_NUM = 0 is 1.0.
The best lambda for PRI_JET_NUM = 1 is 1.0.
The best lambda for PRI_JET_NUM = 2 is 1.0.


<Figure size 640x480 with 0 Axes>

In [26]:
accuracy, precision, recall, f1_score = compute_metrics(y_tr_true, y_tr_pred)
print("Training")
print(accuracy, precision, recall, f1_score)

accuracy, precision, recall, f1_score = compute_metrics(y_dev_true, y_dev_pred)
print("Validation")
print(accuracy, precision, recall, f1_score)

Training
0.74639 0.7040720885137447 0.44981271770655706 0.5489292828685259
Validation
0.74558 0.6979804441195284 0.4478714671044916 0.5456298889166696


In [13]:
def ridge_regression_plot(y_tr, tx_tr, y_dev, tx_dev, lambda_):
    """Ridge regression using normal equations.
    Args:
        y: numpy array of shape (N, 1), N is the number of samples.
        tx: numpy array of shape (N, D), D is the number of features.
        lambda_: scalar.

    Returns:
        w: optimal weights, numpy array of shape(D, 1), D is the number of features.
        loss: scalar
    """
    N, D = tx_tr.shape
    I = np.eye(D)
    w = np.linalg.solve(tx_tr.T @ tx_tr + 2 * N * lambda_ * I, tx_tr.T @ y_tr).reshape(-1, 1)
    train_loss = compute_mse(y_tr, tx_tr, w)
    dev_loss = compute_mse(y_dev, tx_dev, w)

    return w, train_loss, dev_loss

In [14]:
train_losses = []
dev_losses = []
ws = []
y_tr_pred, y_tr_true = np.empty((0, 1)), np.empty((0, 1))
y_dev_pred, y_dev_true = np.empty((0, 1)), np.empty((0, 1))

for i in range(len(tx_train_list)):
    train_losses = []
    dev_losses = []
    w_list = []
    lambda_list = [0]
    # lambda_list = [1e-1]

    y_tr = y_tr_list[i]
    tx_tr_fe = tx_train_list[i]
    y_dev = y_dev_list[i]
    tx_dev_fe = tx_dev_list[i]

    for lambda_ in lambda_list:
        w, train_loss, dev_loss = ridge_regression_plot(
            y_tr, tx_tr_fe,
            y_dev, tx_dev_fe,
            lambda_,
        )
        train_losses.append(train_loss)
        dev_losses.append(dev_loss)
        w_list.append(w)

    # cross_validation_visualization(lambda_list, train_losses, dev_losses, i)
    index = np.argmin(dev_losses)
    best_lambda = lambda_list[index]
    best_w = w_list[index]
    print("The best lambda for PRI_JET_NUM = {} is {}.".format(i, best_lambda))

    y_tr_pred = np.vstack((y_tr_pred, predict_linear(tx_tr_fe, best_w)))
    y_dev_pred = np.vstack((y_dev_pred, predict_linear(tx_dev_fe, best_w)))
    y_tr_true = np.vstack((y_tr_true, y_tr))
    y_dev_true = np.vstack((y_dev_true, y_dev))
    ws.append(best_w)

The best lambda for PRI_JET_NUM = 0 is 0.
The best lambda for PRI_JET_NUM = 1 is 0.
The best lambda for PRI_JET_NUM = 2 is 0.


In [15]:
accuracy, precision, recall, f1_score = compute_metrics(y_tr_true, y_tr_pred)
print("Training")
print(accuracy, precision, recall, f1_score)

accuracy, precision, recall, f1_score = compute_metrics(y_dev_true, y_dev_pred)
print("Validation")
print(accuracy, precision, recall, f1_score)

Training
0.80358 0.7449431258247173 0.6500080159736493 0.6942451082641927
Validation
0.80356 0.7417435486027544 0.6505805089715023 0.6931775584155943


In [33]:
D = tx_te.shape[1]
y_pred = np.empty((0, 1))
for i in range(len(tx_te)):
    pri_jet_num = np.min((2, int(tx_te[i, -1])))
    w = ws[pri_jet_num]

    if pri_jet_num == 0:
        col_mask = np.ones(D, dtype=bool)
        col_mask[[3, 4, 5, 11, 21, 22, 23, 24, 25, 26, 27, 28]] = False
        tx_cleaned = tx_te[i, col_mask].reshape(1, -1)
    elif pri_jet_num == 1:
        col_mask = np.ones(D, dtype=bool)
        col_mask[[3, 4, 5, 11, 25, 26, 27, 28]] = False
        tx_cleaned = tx_te[i, col_mask].reshape(1, -1)
    else:
        tx_cleaned = tx_te[i, :-1].reshape(1, -1)

    tx_cleaned = build_poly(tx_cleaned, degree)
    # tx = ((tx_cleaned-means[pri_jet_num])/(stds[pri_jet_num]+1e-9)).reshape(1, -1)
    tx = ((tx_cleaned-stds[pri_jet_num])/(means[pri_jet_num]-stds[pri_jet_num])).reshape(1, -1)
    y_pred = np.vstack((y_pred, predict_linear(tx, w)))

In [34]:
y_pred[y_pred==0] = -1
y_pred = y_pred.astype(int)

In [35]:
create_csv_submission(ids_te, y_pred, 'submission_linear_regression_cross_validation.csv')